In [53]:
import re
import io
import csv
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
from googletrans import Translator
import unicodedata

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [2]:
def StandardizeCasing(message):
    return message.lower()

In [3]:
def RemoveUsermentions(message):
    regex = re.compile('@[A-Za-z0-9_-]{2,}', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    message = re.sub(regex, '', message)
    return message.strip()
    

In [4]:
def RemoveUrls(message):
    regex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', re.S)
    message = re.sub(regex, '', message)
    return message.strip()

In [5]:
def RemoveEmoji(message):
    #regex = re.compile('\[a-z]*',re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    regex = re.compile(r'\\x[A-Za-z0-9]*', re.S)
    message = re.sub(regex, '', message)
    return message.strip()
    #return message.encode('ascii', 'ignore').decode('ascii')
    

In [6]:
def RemovePunctuation(message):
    regex = re.compile('[,:/\"[\]]', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    message = re.sub(regex, '', message)
    return message.strip()
    

In [7]:
def RemoveDigits(message):
    regex = re.compile('[0-9]+', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    message = re.sub(regex, '', message)
    return message.strip()
    

In [8]:
def RemoveUnicode(message):
    return unicodedata.normalize('NFKD', message).encode('ascii','ignore')
    

In [9]:
def RemoveNewlinewithin(message):
    #regex = re.compile('\x', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    #message = re.sub(regex, '', message)
    #return message.strip()
    return message.replace('\\n', ' ')
    

In [10]:
def removeDots(message):
    regex = re.compile('[.]+', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    message = re.sub(regex, ' ', message)
    return message.strip()

In [47]:
def RemoveSpecialChars(message):
    #regex = re.compile('[?*\"\'!#$^\&~`@]', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    regex = re.compile(r"[-()\"#/@;:<>{}`+=~|.!?,\&]", re.S)
    message = re.sub(regex, '', message)
    return message.strip()

In [12]:
def RemoveRT(message):
    regex = re.compile('rt ', re.S) #remove the @ sign with any trailing letters, digits or _ 2 or more
    message = re.sub(regex, '', message)
    return message.strip()

In [13]:
def RemoveDEmoji(message):
    emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', message)

In [14]:
def Transliterate(message, highlish_dict):
    transmessage = []
    for word in message.split():
        #print(word , ":", trans.get(word))
        if (trans.get(word)!= None):
            transmessage.append(trans[word])
        else:
            transmessage.append(word)
    return " ".join(transmessage)                   
        
    

In [66]:
def RemoveStopWords(message, stop_words):
    tokens = word_tokenize(message)
    filtered_tokens = [word for word in tokens if not word in stop_words]
    message = " ".join(filtered_tokens)
    return message
    

In [1]:
def PreprocessMessage(message, trans):
    message = RemoveEmoji(message)
    message = StandardizeCasing(message)
    message = RemoveUsermentions(message)
    message = RemoveUrls(message)
    message = RemovePunctuation(message)
    message = RemoveDigits(message)
    message = RemoveNewlinewithin(message)
    message = removeDots(message)
    message = RemoveSpecialChars(message)
    message = RemoveRT(message)
    message = Transliterate(message, trans)
    return message

In [69]:
def EngPreprocessMessage(message):
    message = RemoveEmoji(message)
    message = StandardizeCasing(message)
    message = RemoveUsermentions(message)
    message = RemoveUrls(message)
    message = RemovePunctuation(message)
    message = RemoveDigits(message)
    message = RemoveNewlinewithin(message)
    message = removeDots(message)
    message = RemoveSpecialChars(message)
    message = RemoveRT(message)
    return message

In [63]:
with open(file = r".\\data\transliteration\dictionary_hinglish_transliteration.txt", mode = 'r') as hinglish_dict:
        trans = json.load(hinglish_dict)
        print(type(trans))
        print(trans["hai"])
message = "haa jaise tum bhi abhi pshdf"
print(Transliterate(message, trans))

<class 'dict'>
have
haa : ha
jaise : for example
tum : you
bhi : also
abhi : actually
pshdf : None
ha for example you also actually pshdf


In [88]:
messages = []
labels = []


from nltk.corpus import stopwords
stopwords = stopwords.words('english')

with open(file = r".\\data\transliteration\dictionary_hinglish_transliteration.txt", mode = 'r') as hinglish_dict:
    trans = json.load(hinglish_dict)
    
with open(file = r".\\data\transliteration\Hinglish_Profanity_List.csv", mode = 'r') as profane_data:
    reader = csv.reader(profane_data)
    for row in reader:
        trans[row[0]] = row[1]
        
        
with open('.\\data\\source\\messages.csv', newline ='') as messageData:
    reader = csv.reader(messageData)
    for row in reader:
        message = (''.join(row))
        message = PreprocessMessage(message, trans)
        message = RemoveStopWords(message, stopwords)
        messages.append(message)
        
with open('.\\data\\source\\labels.csv', newline ='') as messageData:
    reader = csv.reader(messageData)
    for row in reader:
        label = (''.join(row))
        labels.append(label)



In [89]:
messages = []
#labels = []


from nltk.corpus import stopwords
stopwords = stopwords.words('english')

with open('.\\data\\final\\transmessages.csv', newline ='') as messageData:
    reader = csv.reader(messageData)
    for row in reader:
        message = (''.join(row))
        #message = PreprocessMessage(message, trans)
        message = RemoveStopWords(message, stopwords)
        messages.append(message)

In [90]:
labels

['0',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '0',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '0',
 '0',
 '2',
 '1',
 '0',
 '2',
 '2',
 '2',
 '2',
 '2'

In [73]:
engmessages = []
englabels = []
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

with open('.\\data\\source\\engmessages.csv', newline ='') as messageData:
    reader = csv.reader(messageData)
    for row in reader:
        message = (''.join(row))
        message = EngPreprocessMessage(message)
        message = RemoveStopWords(message, stopwords)
        engmessages.append(message)
        
with open('.\\data\\source\\englabels.csv', newline ='') as messageData:
    reader = csv.reader(messageData)
    for row in reader:
        label = (''.join(row))
        englabels.append(label)


In [91]:
with open('.\\data\\final\\messages_2.csv', mode='w') as tweets:
    writer = csv.writer(tweets, delimiter='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows([messages])
with open('.\\data\\final\\labels_2.csv', mode='w') as tweets:
    writer = csv.writer(tweets, delimiter='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows([labels])


In [75]:
with open('.\\data\\final\\engmessages.csv', mode='w') as tweets:
    writer = csv.writer(tweets, delimiter='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows([engmessages])
with open('.\\data\\final\\englabels.csv', mode='w') as tweets:
    writer = csv.writer(tweets, delimiter='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows([englabels])

In [78]:
len(engmessages)

24784